In [34]:
from pathlib import Path

from cast.moderation import Moderator
from cast.models.moderation import NaiveBayes

In [14]:
SpamFilter.objects.all()

<QuerySet [<SpamFilter: SpamFilter object (1)>]>

In [6]:
sm = SpamFilter(name="naive bayes")
sm.save()

In [8]:
sm.retrain_from_scratch()

In [10]:
mod = Moderator(spamfilter=sm)

TypeError: Moderator.__init__() missing 1 required positional argument: 'model'

In [15]:
Comment.objects.all()

<QuerySet []>

# Load Comments

In [50]:
import json

from collections import Counter

In [24]:
with Path("comments.json").open("r") as f:
    json_comments = json.load(f)

site = Site.objects.first()
attrs = ["name", "title", "comment", "is_public", "is_removed"]
comments = []
for row in json_comments:
    comment = Comment()
    for attr in attrs:
        setattr(comment, attr, row[attr])
    comments.append(comment)

In [52]:
def get_training_data(comments):
    train = []
    for comment in comments:
        label = "ham" if (comment.is_public and not comment.is_removed) else "spam"
        message = SpamFilter.comment_to_message(comment)
        train.append((label, message))
    return train

In [53]:
train = get_training_data(comments)

In [55]:
%%time
model = NaiveBayes().fit(train)

CPU times: user 149 ms, sys: 5.07 ms, total: 154 ms
Wall time: 153 ms


In [56]:
message = train[-1][-1]

In [57]:
model.predict_label(message)

'spam'

In [58]:
message

'Carloslendy  трахнул мать русская порнуха   https://pornuha.pro/  порнуха русская племянники  \r\n \r\n<img src="https://pornuha.pro/picture/Roskoshnaia-koshechka-stimuliruet-pisku-dvumia-vibratorami-i-konchaet.jpg"> \r\n \r\n<a href=http://polyptyq.com/hello-world#comment-102343>порнухи русский ру</a>\r\n<a href=https://havasuembroidery.com/hello-world/#comment-4251>порнушка аниме геи</a>\r\n<a href=http://niwast.sblo.jp/article/186273111.html>порнушка в автобусе</a>\r\n<a href=https://mak-elearning.com/hello-world/#comment-3989>порнуха пьяные русские бабы</a>\r\n<a href=https://ntrinitarias.com/pfizer-inicia-prueba-de-una-pildora-anticovid/#comment-11217>смотреть онлайн бесплатно порнуха русские мамы</a>\r\n<a href=http://martin-wolf-film.de/2014/03/a-shine-co-nyc/#comment-24739>горячая порнуха</a>\r\n<a href=https://coppercitydigital.com/8-reasons-to-redesign-your-website/#comment-387>порнушка мужики</a>\r\n<a href=http://www.niedertor.it/gaestebuch-en.php>смотреть русские порнуху 

In [42]:
model.predict(message)

{'ham': 0.8206452207488063, 'spam': 0.1793547792511937}

In [43]:
model.prior_probabilities

{'ham': 0.7160349854227406, 'spam': 0.2839650145772595}

In [47]:
[l, m for l, m in train if l == "ham"]

SyntaxError: did you forget parentheses around the comprehension target? (503427492.py, line 1)

In [54]:
Counter([l for l, m in train])

Counter({'ham': 122, 'spam': 1593})